<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Train/Test Split and Bias and Variance

_Authors: Joseph Nelson (DC), Kevin Markham (DC); Updated: B Rhodes (DC)_

---

<a id="learning-objectives"></a>
<a id="learning-objectives"></a>
<a id="learning-objectives"></a>
### Learning Objectives
- Define error due to bias and error due to variance.
- Identify the bias-variance trade-off.
- Describe what overfitting and underfitting means in the context of model building.
- Explain problems associated with over- and underfitting.
- Grasp why train/test split is necessary.
- Explore k-folds, LOOCV, and three split methods.

### Lesson Guide
- [Bias and Variance Trade-Off](#bias-and-variance-trade-off)
	- [Bias? Variance?](#bias-variance)
	- [Exploring the Bias-Variance Trade-Off](#exploring-the-bias-variance-tradeoff)
	- [Brain and Body Weight Mammal Data Set](#brain-and-body-weight-mammal-dataset)
	- [Making a Prediction](#making-a-prediction)
- [Making a Prediction From a Sample](#making-a-prediction-from-a-sample)
	- [Let's Try Something Completely Different](#lets-try-something-completely-different)
- [Balancing Bias and Variance](#balancing-bias-and-variance)
- [Train/Test Split](#train-test-split)
	- [Evaluation Procedure #1: Train and Test on the Entire Data Set (Do Not Do This)](#evaluation-procedure--train-and-test-on-the-entire-dataset-do-not-do-this)
	- [Problems With Training and Testing on the Same Data](#problems-with-training-and-testing-on-the-same-data)
	- [Evaluation Procedure #2: Train/Test Split](#evaluation-procedure--traintest-split)
	- [Comparing Test Performance With a Null Baseline](#comparing-test-performance-with-a-null-baseline)
- [K-Folds Cross-Validation](#k-folds-cross-validation)
	- [Leave-One-Out Cross-Validation](#leave-one-out-cross-validation)
	- [Intro to Cross-Validation With the Boston Data](#intro-to-cross-validation-with-the-boston-data)
- [Three-Way Data Split](#three-way-data-split)
	- [Additional Resources](#additional-resources)
- [Summary](#summary)

<a id="bias-and-variance-trade-off"></a>
## Bias and Variance Trade-Off
---

The **bias-variance tradeoff** is widely used in machine learning as a conceptual way of comparing and contrasting different models. We only have a few methods that are able to compare all machine learning models. The others are more mathematical.

**Bias** is error stemming from incorrect model assumptions.
- Example: Assuming data is linear when it has a more complicated structure.

**Variance** is error stemming from being overly sensitive from changes to the training data.
- Example: Using the training set exactly (e.g. 1-NN) for a model results in a completely different model -- even if the training set differs only slightly.


As model complexity **increases**:
- Bias **decreases**. (The model can more accurately model complex structure in data.)
- Variance **increases**. (The model identifies more complex structures, making it more sensitive to small changes in the training data.)

In [ ]:
#You can't have a value taht is zero bias otherwise you don't need to use machine learning 
#as model complexity goes up bias decreases and variance increases 

<a id="bias-variance"></a>
### Bias? Variance?

**Conceptual Definitions**
- **Bias**: How close are predictions to the actual values?
  - Roughly, whether or not our model aims on target.
  - If the model cannot represent the data's structure, our predictions could be consistent, but will not be accurate.
- **Variance**: How variable are our predictions?
  - Roughly, whether or not our model is reliable.
  - We will make slightly different predictions given slightly different training sets.

![](./assets/biasVsVarianceImage.png)

- Visually, we are building a model where the bulls-eye is the goal.
- Each individual hit is one prediction based on our model.
- Critically, the success of our model (low variance, low bias) depends on the training data present.

**Examples:**

- **Linear regression:** Low variance, High bias.
    - If we train with a different subset of the training set, the model will be about the same. Hence, the model has low variance.
    - The resulting model will predict the training points incorrectly (unless they happen to be perfectly linear). Hence, it has high bias.
   

- **Nearest neighbor:** High variance, Low bias.
    - If we train with a different subset of the training set, the model will make predictions very differently. Hence, the model is highly variable.
    - The resulting model will predict every training point perfectly. Hence, it has low bias.

- **K-Nearest neightbor:** Med-high variance, Med-low bias.
    - The model itself is more robust to outliers, so it will make more predictions the same than before. Hence, it has lower variance than 1-NN.
    - The resulting model no longer predicts every point perfectly, since outliers will be mispredicted. So, the bias will be higher than before.


See if you can figure out:

- **High-order polynomial (as compared to linear regression)**

**Expressing bias and variance mathematically:**

It can be helpful understanding these terms by looking at how we can decompose the total error into them mathematically. (We will skip the derivations for now!)

Let's define the error of our predictor as the expected value of our squared error. Note this error is not based on any particular fitted model, but on the family of potential models given a dataset (i.e. all fitted models made from all possible subsets of data).

$$E[(y - \hat{f}(x))^2] = \text{Bias}[\hat{f}(x)]^2 + \text{Var}[\hat{f}(x)] + \sigma^2$$

This states the expected error is based on only three components: **bias**, **variance**, and **irreducible error**.

Breaking the bias and variance down further:

$$\text{Bias}[\hat{f}(x)] = E[\hat{f}(x) - f(x)].$$

- The bias is just the average expected distance between our predictor and actual values.

$$\text{Var}[\hat{f}(x)] = E[\hat{f}(x)^2] - E[\hat{f}(x)]^2.$$

- The variance is how much our predictions vary about the mean. ($E[\hat{f}(x)]$ is our predictor's mean prediction.)

- The irreducible error stems from noise in the problem itself.

### MSE equation in plain english


$$ \text{MSE} = \text{variance} + \text{bias}^2 + \text{irreducible error} $$



**Some common questions:**

From the math above, we can answer a few common questions:

Can a model have high bias given one dataset and low bias for another?
- Yes. If our data is linearly related, for example, it will have low bias on a linear model! However, in general across all datasets very few are accurately described with a linear model. So, in general we say a linear model has high bias and low variance.

Is the MSE for a fitted linear regression the same thing as the bias?
- It's close, but bias does not apply to a specific fitted model. Bias is the expected error of a model no matter what subset of the data it is fit on. This way, if we happen to get a lucky MSE fitting a model on a particular subset of our data, this does not mean we will have a low bias overall.

<a id="exploring-the-bias-variance-tradeoff"></a>
### Exploring the Bias-Variance Trade-Off

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Specific Imports
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score

# model saving imports
import pickle
from joblib import dump, load

#import patsy


# Allow plots to appear in the notebook.
%matplotlib inline

ModuleNotFoundError: No module named 'joblib'

<a id="brain-and-body-weight-mammal-dataset"></a>
### Brain and Body Weight Mammal Data Set

This is a [data set](http://people.sc.fsu.edu/~jburkardt/datasets/regression/x01.txt) of the average weight of the body (in kg) and the brain (in g) for 62 mammal species. We'll use this dataset to investigate bias vs. variance. Let's read it into Pandas and take a quick look:

In [ ]:
path = 'data/mammals.txt'
cols = ['brain','body']
mammals = pd.read_table(path, sep='\t', names=cols, header=0)
mammals.head()

In [ ]:
mammals.describe()

We're going to focus on a smaller subset in which the body weight is less than 200 kg.

In [ ]:
# Only keep rows in which the body weight is less than 200 kg.
mammals = mammals[mammals.body < 200]
mammals.shape

We're now going to pretend that there are only 51 mammal species in existence. In other words, we are pretending that this is the entire data set of brain and body weights for **every known mammal species**.

Let's create a scatterplot (using [Seaborn](http://stanford.edu/~mwaskom/software/seaborn/)) to visualize the relationship between brain and body weight:

In [ ]:
sns.lmplot(x='body', y='brain', data=mammals, ci=None, fit_reg=False);
plt.xlim(-10, 200);
plt.ylim(-10, 250);

There appears to be a relationship between brain and body weight for mammals.

<a id="making-a-prediction"></a>
### Making a Prediction

<a id="linear-regression-quick-review"></a>
#### Linear Regression: A Quick Review

![](./assets/linear-residuals.png)

Now let's pretend that a **new mammal species** is discovered. We measure the body weight of every member of this species we can find and calculate an **average body weight of 100 kgs**. We want to **predict the average brain weight** of this species (rather than measuring it directly). How might we do this?

In [ ]:
sns.lmplot(x='body', y='brain', data=mammals, ci=None);
plt.xlim(-10, 200);
plt.ylim(-10, 250);

We drew a straight line that appears to best capture the relationship between brain and body weight. So, we might predict that our new species has a brain weight of about 45 g, as that's the approximate y value when x=100.

This is known as a "linear model" or a "linear regression model."

<a id="making-a-prediction-from-a-sample"></a>
## Making a Prediction From a Sample

Earlier, we assumed that this dataset contained every known mammal species. That's very convenient, but **in the real world, all you ever have is a sample of data**. This may sound like a contentious statement, but the point of machine learning is to generalize from a sample to the population. If you already have data for the entire population, then you have no need for machine learning -- you can apply statistics directly and get optimal answers!

Here, a more realistic situation would be to only have brain and body weights for (let's say) half of the 51 known mammals.

When that new mammal species (with a body weight of 100 kg) is discovered, we still want to make an accurate prediction for its brain weight, but this task might be more difficult, as we don't have all of the data we would ideally like to have.

Let's simulate this situation by assigning each of the 51 observations to **either universe 1 or universe 2**:

In [ ]:
# Set a random seed for reproducibility.
np.random.seed(12345)

# Randomly assign every observation to either universe 1 or universe 2.
mammals['universe'] = np.random.randint(1, 3, len(mammals))
mammals.head()

**Important:** We only live in one of the two universes. Both universes have 51 known mammal species, but each universe knows the brain and body weight for different species.

We can now tell Seaborn to create two plots in which the left plot only uses the data from **universe 1** and the right plot only uses the data from **universe 2**:

In [ ]:
# col='universe' subsets the data by universe and creates two separate plots.
sns.lmplot(x='body', y='brain', data=mammals, ci=None, col='universe');
plt.xlim(-10, 200);
plt.ylim(-10, 250);

The line looks pretty similar between the two plots, despite the fact that they used separate samples of data. In both cases, we would predict a brain weight of about 45 g.

It's easier to see the degree of similarity by placing them on the same plot:

In [ ]:
# hue='universe' subsets the data by universe and creates a single plot.
sns.lmplot(x='body', y='brain', data=mammals, ci=None, hue='universe');
plt.xlim(-10, 200);
plt.ylim(-10, 250);

So, what was the point of this exercise? This was a visual demonstration of a high-bias, low-variance model.

- It's **high bias** because it doesn't fit the data particularly well.
- It's **low variance** because it doesn't change much depending on which observations happen to be available in that universe.

<a id="lets-try-something-completely-different"></a>
### Let's Try Something Completely Different

What would a **low bias, high variance** model look like? Let's try polynomial regression with an eighth-order polynomial.

**Question**: is an 8th order model more or less complex than a linear model?

In [ ]:
sns.lmplot(x='body', y='brain', data=mammals, ci=None, col='universe', order=8);
plt.xlim(-10, 200);
plt.ylim(-10, 250);

- It's **low bias** because the models match the data effectively.
- It's **high variance** because the models are widely different, depending on which observations happen to be available in that universe. (For a body weight of 100 kg, the brain weight prediction would be 40 kg in one universe and 0 kg in the other!)

In [ ]:
sns.lmplot(x='body', y='brain', data=mammals, ci=None, hue='universe', order=8);
plt.xlim(-10, 200);
plt.ylim(-10, 250);

<a id="balancing-bias-and-variance"></a>
## Balancing Bias and Variance
Can we find a middle ground?

Perhaps we can create a model that has **less bias than the linear model** and **less variance than the eighth order polynomial**?

Let's try a second order polynomial instead:

In [ ]:
sns.lmplot(x='body', y='brain', data=mammals, ci=None, col='universe', order=2);
plt.xlim(-10, 200);
plt.ylim(-10, 250);

In [ ]:
sns.lmplot(x='body', y='brain', data=mammals, ci=None, hue='universe', order=2);
plt.xlim(-10, 200);
plt.ylim(-10, 250);

This seems better. In both the left and right plots, **it fits the data well, but not too well**.

This is the essence of the **bias-variance trade-off**: You are seeking a model that appropriately balances bias and variance and thus will generalize to new data (known as "out-of-sample" data).

We want a model that best balances bias and variance. It
should match our training data well (moderate bias) yet be low variance for out-of-sample data (moderate variance).

- Training error as a function of complexity.
- **Question:** Why do we even care about variance if we know we can generate a more accurate model with higher complexity?

### Can we obtain a zero-bias, zero-variance model?

No! If there is any noise in the data-generating process, then a zero-variance model would not be learning from the data. Additionally, a model only has zero bias if the true relationship between the target and the features is hard-coded into it. If that were the case, you wouldn't be doing machine learning -- it would be similar to trying to predict today's temperature by using today's temperature!

---

<a id="train-test-split"></a>
## Train-test-split

For the rest of the lab, we will look at three evaluation procedures for predicting model out-of-sample accuracy:

1. **Train on the entire dataset** should never be done to estimate model accuracy on out-of-sample data! After all, training error can be made arbitrarily small or large. You might train on the entire dataset as the very last step when a model is chosen, hoping to make the final model as accurate as possible. Or, you could use this to estimate the degree of overfitting.
2. **Train-test-split** is useful if cross-validation is not practical (e.g. it takes too long to train). It is also useful for computing a quick confusion matrix. You could also use this as a final step after the model is finalized (often called evaluating the model against a **validation set**).
3. **Cross-validation** is the gold standard for estimating accuracy and comparing accuracy across models.
4. **Three-way split** combines cross-validation and the train-test-split. It takes an initial split to be used as a final validation set, then uses cross-validation on the rest.

We run into a problem when powerful models can perfectly fit the data on which they are trained. These models are **low bias** and **high variance**. However, we can't observe the variance of a model directly, because we only know how it fits the data we have rather than all potential samples.

**Solution:** Create a procedure that **estimates** how well a model is likely to perform on out-of-sample data and use that to choose between models.

- Before, we have been splitting the data into a **single training group** and a **single test group**.

- Now, to estimate how well the model is likely to perform on out-of-sample data, we will create **many training groups** and **many test groups** then fit **many models**.

**Note:** These procedures can be used with **any machine learning model**.


**The Holdout Method: Train/Test Split**
- **Training set**: Used to train the classifier.
- **Testing set**: Used to estimate the error rate of the trained classifier.
- **Advantages**: Fast, simple, computationally inexpensive.
- **Disadvantages** Eliminates data, imperfectly splits.

<a id="evaluation-procedure--train-and-test-on-the-entire-dataset-do-not-do-this"></a>
### Evaluation Procedure #1: Train and Test on the Entire Data Set (Do Not Do This)

1. Train the model on the **entire data set**.
2. Test the model on the **same data set** and evaluate how well we did by comparing the **predicted** response values with the **true** response values.

**Question**: While the warning says do not do this. When might we want to train using the entire dataset?

#### Load in the Boston data.

*Note*: `sklearn` provides a number of "toy" datasets, the Boston housing data is just one example. You can see more here: `sklearn` [toy data sets](https://scikit-learn.org/stable/datasets/index.html#toy-datasets). These are useful to practice with since there are many tutorials and examples on the internet to compare your results against or to use for guidance.

In [5]:
import pandas as pd
import numpy as np

boston = load_boston()

In [6]:
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

#### Create X and y variable to stores the feature matrix and response from the Boston data.

In [7]:
# Create a DataFrame for both parts of data; don't forget to assign column names.
boston_data_df = pd.DataFrame(boston.data, columns=boston.feature_names)
target_ser = pd.DataFrame(boston.target, columns=['MEDV'])

#### Concatenate y and X, then overwrite the Boston variable.

In [8]:
# Concatenate 
boston_df = pd.concat([boston_data_df, target_ser], axis=1)

In [9]:
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### Perform basic EDA to make sure the data are in order.

In [ ]:
#checking missing values

In [10]:
boston_df.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

In [11]:
boston_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null float64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null float64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
MEDV       506 non-null float64
dtypes: float64(14)
memory usage: 55.4 KB


In [12]:
boston_df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


#### Prepare a feature matrix (X) and response (y)  for scikit-learn.

In [13]:
# create feature matrix (X)
# X is out feature values so we want all of the columns except MEDV
feature_cols = boston_df.columns.drop(['MEDV'])
X = boston_df[feature_cols]

# create response vector (y)
y = boston_df['MEDV']

#### Import linear regression, instantiate, fit, and preview predictions.

In [15]:
# Instantiate the model.
lr = LinearRegression()

# Train the model on the entire data set.
lr.fit(X, y)

# Predict the response values for the observations in X ("test the model").
# only show first 10 predictions
lr.predict(X)[:10]

array([30.00384338, 25.02556238, 30.56759672, 28.60703649, 27.94352423,
       25.25628446, 23.00180827, 19.53598843, 11.52363685, 18.92026211])

#### Store the predicted response values.

In [16]:
y_pred = lr.predict(X)

To evaluate a model, we also need an **evaluation metric:**

- A numeric calculation used to **quantify** the performance of a model.
- The appropriate metric depends on the **goals** of your problem.

The most common choices for regression problems are:

- **R-squared**: The percentage of variation explained by the model (a "reward function," as higher is better).
- **Mean squared error**: The average squared distance between the prediction and the correct answer (a "loss function," as lower is better).

In this case, we'll use mean squared error because it is more interpretable in a predictive context.

#### Compute mean squared error using a function from `metrics`.

In [17]:
mse = metrics.mean_squared_error(y, y_pred)
print(f'MSE = {mse}')

MSE = 21.894831181729202


What units are this in?

This is known as the **training mean squared error** because we are evaluating the model based on the same data we used to train the model.

We can also calculate the **RMSE** or root mean squared error.

In [ ]:
# value square

In [19]:
print(f'RMSE = {np.sqrt(mse)}')

RMSE = 4.679191295697281


**Question**: What are the units for RMSE in this problem?

<a id="problems-with-training-and-testing-on-the-same-data"></a>
### Problems With Training and Testing on the Same Data

- Our goal is to estimate likely performance of a model on **out-of-sample data**.
- But, maximizing the training mean squared error rewards **overly complex models** that won't necessarily generalize.
- Unnecessarily complex models **overfit** the training data.
    - They will do well when tested using the in-sample data.
    - They may do poorly with out-of-sample data.
    - They learn the "noise" in the data rather than the "signal."
    - From Quora: [What is an intuitive explanation of overfitting?](http://www.quora.com/What-is-an-intuitive-explanation-of-overfitting/answer/Jessica-Su)

**Thus, the training MSE is not a good estimate of the out-of-sample MSE.**

<a id="evaluation-procedure--traintest-split"></a>
### Evaluation procedure #2: Train/Test Split

1. Split the data set into two pieces: a **training set** and a **testing set**.
2. Train the model on the **training set**.
3. Test the model on the **testing set** and evaluate how well we did.

Often a good rule-of-thumb is 70% training & 30% test, but this can vary based on the size of your dataset. For example, with a small dataset you would need to use as much training data as possible (in return, your test accuracy will be more variable).

*Note: sklearn's `train_test_split()` method defaults to a 75/25 split for training vs testing data.*

What does this accomplish?

- Models can be trained and tested on **different data** (We treat testing data like out-of-sample data).
- Response values are known for the testing set and thus **predictions can be evaluated**.

This is known as the **testing mean squared error** because we are evaluating the model on an independent "test set" that was not used during model training.

**The testing MSE is a better estimate of out-of-sample performance than the training MSE.**

#### Understanding the `train_test_split` Function

In [20]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [21]:
# Before splitting
print(f'Features before and after splitting:')
print(f'Before split: {X.shape}')

# After splitting
print(f'Train split: {X_train.shape}')
print(f'Test split: {X_test.shape}')

Features before and after splitting:
Before split: (506, 13)
Train split: (379, 13)
Test split: (127, 13)


In [22]:
# spot check the size of our train and test data.
print(f'Train proportion: {round(X_train.shape[0]/X.shape[0],2)}')
print(f'Test proportion: {round(X_test.shape[0]/X.shape[0],2)}')

Train proportion: 0.75
Test proportion: 0.25


In [23]:
# Recall that (1,) is a tuple. 
# The trailing comma distinguishes it as being a tuple, not an integer.

# Before splitting
print(f'Features before and after splitting:')
print(f'Before split: {y.shape}')

# After splitting
print(f'Train split: {y_train.shape}')
print(f'Test split: {y_test.shape}')

Features before and after splitting:
Before split: (506,)
Train split: (379,)
Test split: (127,)


In [24]:
# spot check the size of our train and test data.
print(f'Train proportion: {round(y_train.shape[0]/y.shape[0],2)}')
print(f'Test proportion: {round(y_test.shape[0]/y.shape[0],2)}')

Train proportion: 0.75
Test proportion: 0.25


In [25]:
# Display the first 5 rows of X_train
X_train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
413,28.65580,0.0,18.10,0.0,0.597,5.155,100.0,1.5894,24.0,666.0,20.2,210.97,20.08
150,1.65660,0.0,19.58,0.0,0.871,6.122,97.3,1.6180,5.0,403.0,14.7,372.80,14.10
246,0.33983,22.0,5.86,0.0,0.431,6.108,34.9,8.0555,7.0,330.0,19.1,390.18,9.16
137,0.35233,0.0,21.89,0.0,0.624,6.454,98.4,1.8498,4.0,437.0,21.2,394.08,14.59
147,2.36862,0.0,19.58,0.0,0.871,4.926,95.7,1.4608,5.0,403.0,14.7,391.71,29.53


In [26]:
# Display the first 5 rows of y_train
y_train.head()

413    16.3
150    21.5
246    24.3
137    17.1
147    14.6
Name: MEDV, dtype: float64

![train_test_split](./assets/train_test_split.png)

#### Understanding the `random_state` Parameter

The `random_state` is a pseudo-random number that allows us to reproduce our results every time we run them. However, it makes it impossible to predict what are exact results will be if we chose a new `random_state`.

`random_state` is very useful for testing that your model was made correctly since it provides you with the same split each time. However, make sure you remove it if you are testing for model variability!

In [27]:
# WITHOUT a random_state parameter:
#  (If you run this code several times, you get different results!)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Print the first element of each object.
X_train.head(1)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
330,0.04544,0.0,3.24,0.0,0.46,6.144,32.2,5.8736,4.0,430.0,16.9,368.57,9.09


In [28]:
# WITH a random_state parameter:
#  (Same split every time! Note you can change the random state to any integer.)

# Fill in your own value for random_state, i.e. replace the ? with any integer you want.
# Try to pick an integer different than the instructor
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 17)

# Print the first element of each object.
print(X_train.head(1))
print(X_test.head(1))
print(y_train.head(1))
print(y_test.head(1))

        CRIM    ZN  INDUS  CHAS    NOX     RM  AGE     DIS  RAD    TAX  \
251  0.21409  22.0   5.86   0.0  0.431  6.438  8.9  7.3967  7.0  330.0   

     PTRATIO       B  LSTAT  
251     19.1  377.07   3.59  
        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
191  0.06911  45.0   3.44   0.0  0.437  6.739  30.8  6.4798  5.0  398.0   

     PTRATIO       B  LSTAT  
191     15.2  389.71   4.69  
251    24.8
Name: MEDV, dtype: float64
191    30.5
Name: MEDV, dtype: float64


#### Alternative Approach to Creating `X` and `y`: Patsy

We will make one more modification. Patsy is a library that allows you to quickly perform simple data transformations in a style similar to R.

Rather than manually creating X and y, we will use the `.dmatricies()` function from Patsy to create the matricies and explore the effect of changing features on training and testing error.

In [29]:
# We're not using this today, but you may see it out in the wild so it's here for reference.
# patsy.dmatrices("MEDV ~ AGE + RM", data=boston_df, return_type="dataframe")


#### Step 1: Split X and y into training and testing sets (using `random_state` for reproducibility).

In [30]:
X = boston_df[['AGE', 'RM']]
y = boston_df['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=99)

In [31]:
X_train.tail()

,AGE,RM
201,38.4,6.162
168,96.1,6.319
185,68.8,6.153
35,68.2,5.933
129,94.7,5.637


#### Step 2: Train the model on the training set.

In [32]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

#### Step 3: Test the model on the testing set and check the accuracy.

In [33]:
y_pred = lr.predict(X_test)

# Training MSe
mse_train = mean_squared_error(y_train, lr.predict(X_train))

# Testing MSE
mse_test = mean_squared_error(y_test, y_pred)

print(f' Training MSE: {mse_train}')
print(f' Testing MSE:  {mse_test}')

# Training and TestingRMSE
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
print()
print(f' Training RMSE: {rmse_train}')
print(f' Testing RMSE:  {rmse_test}')

 Training MSE: 38.969055787686834
 Testing MSE:  42.001752214868425

 Training RMSE: 6.242519986967349
 Testing RMSE:  6.480875883309943


![Bias-variance tradeoff](./assets/bias_variance.png)

#### Go back to Step 1 and try adding new variables and transformations.

- **Training error**: Decreases as model complexity increases (lower value of k).
- **Testing error**: Is minimized at the optimum model complexity.

### Saving a Model
Building and tweaking a model can take a lot of time and might occur over hours or days even. So you don't have to rebuild things from scratch each time you can actually save the current state of your model.

`sklearn` offers ways to do this and you can seem some basic guidance at this link: [Model Persistance in sklearn](https://scikit-learn.org/stable/modules/model_persistence.html)

**Note**. This can also be used to more extensively save a range of data. You may consider saving your training data and other snapshots of data that let you validate the model build.

In [34]:
#import pickle

# Using pickle

# Save the model
pickle.dump(lr, open("lr_pickle.pkl", 'wb'))

# Restore the model
lr_from_pickle = pickle.load(open('lr_pickle.pkl', 'rb'))

# Use the loaded model to make predictions
y_pred_from_pickle = lr_from_pickle.predict(X_test)

# Compute Test RMSE for both models
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_lr_pickle = np.sqrt(mean_squared_error(y_test, y_pred_from_pickle))

print(f'LR RMSE: {rmse_lr}')
print(f'LR Pickle RMSE: {rmse_lr_pickle}')

LR RMSE: 6.480875883309943
LR Pickle RMSE: 6.480875883309943


In [35]:
# Try joblib, which is more efficient at handling large numpy arrays of data.
#from joblib import dump, load

# Save the model as a pickle file
dump(lr, 'filename.pkl')

# Load fthe model from the file
lr_from_joblib = load('filename.pkl')

# Use the model to make predictions
y_pred_from_joblib = lr_from_joblib.predict(X_test)

# Testing RMSE
# Compute Test RMSE for both models
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_lr_joblib = np.sqrt(mean_squared_error(y_test, y_pred_from_joblib))

print(f'LR RMSE: {rmse_lr}')
print(f'LR Pickle RMSE: {rmse_lr_joblib}')

NameError: name 'dump' is not defined

**WARNING: Don't just download and load random pickle files. They aren't secure**

<a id="comparing-test-performance-with-a-null-baseline"></a>
### Comparing Test Performance With a Null Baseline

When interpreting the predictive power of a model, it's best to compare it to a baseline using a dummy model, sometimes called a ZeroR model or a baseline model. A dummy model is simply using the mean, median, or most common value as the prediction. This forms a benchmark to compare your model against and becomes especially important in classification where your null accuracy might be 95 percent.

For example, suppose your dataset is **imbalanced** -- it contains 99% one class and 1% the other class. Then, your baseline accuracy (always guessing the first class) would be 99%. So, if your model is less than 99% accurate, you know it is worse than the baseline. Imbalanced datasets generally must be trained differently (with less of a focus on accuracy) because of this.

You can alternatively use simple models to achieve baseline results, for example nearest neighbors or a basic unigram bag of words for text data.

#### Examine the baseline mean squared error using a null model.
How does this compare to what we achieved with linear regression. Is our model making an actual improvement?

In [36]:
# Brute force - way to build the null model
# create an array the same size as y_test and fill with the mean value of y_test
y_mean = np.full_like(y_test, y_train.mean())

# compute the MSE for the null model, aka the mean
mse_null = mean_squared_error(y_test, y_mean)
rmse_null = np.sqrt(mse_null)
print(f'Null MSE: {mse_null}')
print(f'Null RMSE: {rmse_null}')


Null MSE: 93.1148669975404
Null RMSE: 9.649604499539885


##### Dummy Models in Scikit-learn
Luckily `sklearn` provides an alternative approach to building baseline models, which comes in handy, because it's very much like building any other model. The dummy algorithms are located in the `dummy` module or `sklearn.dummy`.

Here we'll use a dummy regessor which we bring in using:

`from sklearn.dummy import DummyRegressor`

In [37]:
from sklearn.dummy import DummyRegressor

# Follow the same steps as a real model
# Instantiate
dr = DummyRegressor(strategy='mean')

# Fit or "Train" the dummy regressor
dr.fit(X_train, y_train)

# save the "predictions"
y_pred_dummy = dr.predict(X_test)

# Test MSE for dummy
mse_dum = mean_squared_error(y_test, y_pred_dummy)

# Test RMSE for dummy
rmse_dum = np.sqrt(mse_dum)

print(f'Dummy RMSE: {rmse_dum}')

Dummy RMSE: 9.649604499539885


#### Exercise (~20min)
Build a linear regression model for the Las Vegas Trip Advisor data.

The data is located here: `path = './data/LasVegasTripAdvisorReviews-Dataset.csv'`

Carry out the following steps in the cells below:
1. Load the data set using `.read_csv()`
2. Verify the data was loaded correctly
3. Assign `X` and `y` variables using `Hotel Stars` as your feature and `Score` as your target.
4. Split the data into Train and Test data sets. Use `random_state=99`.
5. Build a linear regression model. Use the standard steps we have demonstrated in previous lessons and in this notebook. You may refer to previous examples in this notebook for reference.
6. Compute the RMSE for the training data and testing data.
7. Build a baseline model.
8. Compute the RMSE for a baseline model using the mean. Remember: here you want to calculate RMSE for the test set.

**Load the dataset using `.read_csv()` and the path above. Verify the data loaded properly.**

In [ ]:
# Load the data
path = './data/LasVegasTripAdvisorReviews-Dataset.csv'

In [ ]:
# Quick exploration of the data


In [ ]:
# Generate summary statistics


In [ ]:
# Look at the column names


**Create X and y using `Hotel Stars` and `Score` as the feature and target, respectively.**

In [ ]:
# Assign X and y - 
# Note: Pay attention to the dimensions (aka shape) of your data.


**Split the data into train and test data sets. Use `random_state=99`**

In [ ]:
# Split the data into train and test data sets


**Build a linear regression model and use it to make predictions.**

In [ ]:
# Build a linear regression model


**Evaluate the train and test RMSE and print your results**

In [ ]:
# Evaluate the train and test RMSE - print your results



**Build a baseline model and compute the RMSE for a baseline model using the mean. Print your result.** 

In [ ]:
# Build a baselne model using the mean



**Compare the linear regression to the baseline model**

**Answer here:**

<a id="k-folds-cross-validation"></a>
## K-Folds Cross-Validation
---

Train/test split provides us with helpful tool, but it's a shame that we are tossing out a large chunk of our data for testing purposes.

**How can we use the maximum amount of our data points while still ensuring model integrity?**

1. Split our data into a number of different pieces (folds).
2. Train using `k-1` folds for training and a different fold for testing.
3. Average our model against EACH of those iterations.
4. Choose our model and TEST it against the final fold.
5. Average all test accuracies to get the estimated out-of-sample accuracy.

Although this may sound complicated, we are just training the model on k separate train-test-splits, then taking the average of the resulting test accuracies!

![](./assets/cross_validation_diagram.png)

<a id="leave-one-out-cross-validation"></a>
### Leave-One-Out Cross-Validation

A special case of k-fold cross-validation is leave-one-out cross-validation. Rather than taking 5–10 folds, we take a fold of size `n-1` and leave one observation to test.

Typically, 5–10 fold cross-validation is recommended.

<a id="intro-to-cross-validation-with-the-boston-data"></a>
### Intro to Cross-Validation With the Boston Data

#### Create a cross-valiation with five folds.
`KFold()` provides train/test indices to split data in train/test sets. It splits the dataset into k consecutive folds (without shuffling by default). Each fold is then used once as a validation while the k - 1 remaining folds form the training set.

In [ ]:
# from sklearn.model_selection import KFold
# Build a K-fold with 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=5432)

**Let's rebuild a model for the boston housing data using `AGE` only.**

In [ ]:
# Lets assign the Boston housing variables again, since we used X & y for other things since then.
X = boston_df[['AGE']]
y = boston_df['MEDV']

# compute the scores - note we're using the linear regression object we instantiated earlier
mse_cross = np.mean(-cross_val_score(lr, X, y, cv=kf, scoring='neg_mean_squared_error'))
rmse_cross = np.sqrt(mse_cross)
r_squared_cross = np.mean(cross_val_score(lr, X, y, cv=kf))

print(f'Cross Validated MSE: {mse_cross}')
print(f'Cross Validated RMSE: {rmse_cross}')
print(f'Cross Validated R-squared: {r_squared_cross}')


**Repeat just using `AGE` and `RM` as the feature.**

In [ ]:
# Lets assign the Boston housing variables again, since we used X & y for other things since then.
X = boston_df[['AGE', 'RM']]
y = boston_df['MEDV']

# compute the scores - note we're using the linear regression object we instantiated earlier
mse_cross = np.mean(-cross_val_score(lr, X, y, cv=kf, scoring='neg_mean_squared_error'))
rmse_cross = np.sqrt(mse_cross)
r_squared_cross = np.mean(cross_val_score(lr, X, y, cv=kf))

print(f'Cross Validated MSE: {mse_cross}')
print(f'Cross Validated RMSE: {rmse_cross}')
print(f'Cross Validated R-squared: {r_squared_cross}')


**Let's look under the hood for a minute**

In [ ]:
# run this once as is and then swap kf for an integer between 5 & 10
-cross_val_score(lr, X, y, cv=kf, scoring='neg_mean_squared_error')

Notice the range of values above. See how they differ when you substitue an integer for our KFold object, `kf`. Ideally we want our errors to be narrowly distributed. While the average error might be relatively low, it's better if it's low and the range of component errors doesn't vary widely.

**Take a look at the results for each fold**

In [ ]:
mse_values = []
scores = []
n = 0

print("~~~~ CROSS VALIDATION each fold ~~~~")
for train_index, test_index in kf.split(X, y):
    lr = LinearRegression().fit(X.iloc[train_index], y.iloc[train_index])
    
    mse_values.append(metrics.mean_squared_error(y.iloc[test_index], lr.predict(X.iloc[test_index])))
    scores.append(lr.score(X, y))
    
    n += 1
    
    print(f'Model {n}')
    print(f'MSE: {mse_values[n-1]}')
    print(f'R2: {scores[n-1]}\n')


print("~~~~ SUMMARY OF CROSS VALIDATION ~~~~")
print(f'Mean of MSE for all folds: {np.mean(mse_values)}')
print(f'Mean of R2 for all folds: {np.mean(scores)}')

**Now that we've found the best model using `AGE` and `RM` we can fit our model using the entire dataset.**

Then we would take that final model and put it into production.

In [ ]:
X = boston_df[['AGE', 'RM']]
y = boston_df['MEDV']

lr = LinearRegression()
lr.fit(X, y)
y_pred = lr.predict(X)

y_pred_series = pd.Series(y_pred[:10])

In [ ]:
y_df = pd.concat([y[:10],y_pred_series], axis=1)
y_df.columns = ['MEDV', 'y_pred']
y_df

While the cross-validated approach here generated more overall error, which of the two approaches would predict new data more accurately — the single model or the cross-validated, averaged one? Why?



**Answer**: 

....

###### Exercise
Use cross validation for the car data set.

For this exercise you'll do the following:
1. Load the `mtcars.csv` data set. The data is located here: `path = './data/mtcars.csv'`
2. Examine the shape of the data and why doing a standard train-test split would be a problem.
3. Set up the `X` and `y` using `vs` and `cyl` as features and `mpg` as the target.
4. Instantiate a new linear regressor and a 5-fold cross validation on the data. Be sure to shuffle the data and use `random_state=1234`.
5. Calculate the cross validated MSE, RMSE, and R-squared. Print your results.
6. Repeat steps 2-5 except use `disp` and `qsec` for features. Keep `mpg` as the target.
7. Compare the metrics and determine which model will perform better.
8. Once you have determined your "best" model, apply it to the full dataset and make predictions.
9. Build a dataframe to display the first 5 predictions alongside the original data (i.e. the original `y`).


**Load the `mtcars.csv` data set.**

In [ ]:
# Load the data


**Examine the shape of the data and why doing a standard train-test split would be a problem.**

In [ ]:
# Examine the shape


**Answer**: The dataset is very small and a 75/25 split would use only 24 observations to train the model. That's a very small amount of data. It's better to use k-fold cross validation since it will use all the data to give us more robust results.

**Set up the `X` and `y` using `vs` and `cyl` as features and `mpg` as the target.**

In [ ]:
# Assign the X and y variables


**Instantiate a new linear regressor and a 5-fold cross validation on the data. Be sure to shuffle the data and use `random_state=1234`.**

In [ ]:
# Set up a linear regression and 5-fold cross validation


**Calculate the cross validated MSE, RMSE, and R-squared. Print your results.**

In [ ]:
# Calculate the cross validated MSE, RMSE, and R-squared. Print your results.


**Repeat steps 2-5 except use `disp` and `qsec` for features. Keep `mpg` as the target.**

In [ ]:
# Repeat steps 2-5 except use disp and qsec for features. Keep mpg as the target.


**Compare the metrics and determine which model will perform better.**

**Answer here- briefly explain why.**


**Once you have determined your "best" model, apply it to the full dataset and make predictions.**

In [ ]:
# Build a model using the parameters of your best model and fit it to the entire data set and make predictions.


**Build a dataframe to display the first 5 predictions alongside the original data (i.e. the original `y`).**

In [ ]:
# Build a dataframe of the original y and the predicted y and display the first 5 rows.


<a id="three-way-data-split"></a>
## Three-Way Data Split
---

The most common workflow is actually a combination of train/test split and cross-validation. We take a train/test split on our data right away and try not spend a lot of time using the testing data set. Instead, we take our training data and tune our models using cross-validation. When we think we are done, we do one last test on the testing data to make sure we haven't accidently overfit to our training data.

**If you tune hyperparameters via cross-validation, you should never use cross-validation on the same dataset to estimate OOS accuracy!** Using cross-validation in this way, the entire dataset was used to tune hyperparameters. So, this invalidates our condition above -- where we assumed the test set is a pretend "out-of-sample" dataset that was not used to train our model! So, we would expect the accuracy on this test set to be artificially inflated as compared to actual "out-of-sample" data.

Even with good evaluation procedures, it is incredible easy to overfit our models by including features that will not be available during production or leak information about our testing data in other ways.

![](./assets/Train-Test-Split-CV.png)

- If model selection and true error estimates are to be computed simultaneously, three disjointed data sets are best.
    - **Training set**: A set of examples used for learning – what parameters of the classifier?
    - **Validation set**: A set of examples used to tune the parameters of the classifier.
    - **Testing set**: A set of examples used ONLY to assess the performance of the fully trained classifier.
- Validation and testing must be separate data sets. Once you have the final model set, you cannot do any additional tuning after testing.

1. Divide data into training, validation, and testing sets.
2. Select architecture (model type) and training parameters (k).
3. Train the model using the training set.
4. Evaluate the model using the training set.
5. Repeat 2–4 times, selecting different architectures (models) and tuning parameters.
6. Select the best model.
7. Assess the model with the final testing set.

<a id="additional-resources"></a>
<a id="additional-resources"></a>
### Additional Resources
- [Bias Variance](http://scott.fortmann-roe.com/docs/BiasVariance.html)
- University of Washington [slides](https://courses.cs.washington.edu/courses/cse546/12wi/slides/)

<a id="summary"></a>
### Summary

In this lab, we compared four methods of estimating model accuracy on out-of-sample data. Throughout your regular data science work, you will likely use all four at some point:

1. **Train on the entire dataset**
2. **Train-test-split**
3. **Cross-validation**
4. **Three-way split**